In [ ]:

class IntCode:
    def __init__(self, sequence):
        self.seq_copy = sequence
        self.sequence = {index:val for index, val in enumerate(self.seq_copy.copy())}
        self.index = 0
        self.relative_base = 0
        self.input_val = []
    
    def reset(self):
        self.index = 0
        self.relative_base = 0
        self.input_val = []
        self.sequence = {index:val for index, val in enumerate(self.seq_copy.copy())}

    @staticmethod
    def read_mod(mods, offset):
            return mods[offset] if 0 <= offset < len(mods) else 0
    
    def read_val(self, mods, offset):
        val = self.sequence.get(self.index + offset, 0)

        mod = self.read_mod(mods, offset)
        if mod == 0:
            return self.sequence.get(val, 0)
        elif mod == 1:
            return val
        elif mod == 2:
            return self.sequence.get(self.relative_base + val, 0)

    def write_val(self, new_value, mods, offset):
        val = self.sequence.get(self.index + offset, 0)

        mod = self.read_mod(mods, offset)
        if mod == 0:
            self.sequence.update({val: new_value})
        elif mod == 2:
            self.sequence.update({self.relative_base + val: new_value})

    def run(self):
        while(True):
            opcode = self.sequence.get(self.index, 0)
            val = opcode % 100
            mods = list(map(int, str(opcode)))[:-2]
            mods.reverse()
            self.index += 1

            if val in [1, 2, 7, 8]:
                op1 = self.read_val(mods, 0)
                op2 = self.read_val(mods, 1)
                if val == 1:
                    res = op1 + op2
                elif val == 2:
                    res = op1 * op2
                elif val == 7:
                    res = 1 if op1 < op2 else 0
                elif val == 8:
                    res = 1 if op1 == op2 else 0

                self.write_val(res, mods, 2)
                self.index += 3

            elif val == 3:
                if not self.input_val:
                    self.index -= 1 # since we added one earlier, we need to remove it before breaking
                    break
                    
                input_val = self.input_val.pop(0)
                self.write_val(input_val, mods, 0)

                self.index += 1

            elif val in [4, 9]:
                op1 = self.read_val(mods, 0)
                if val == 4:
                    yield op1
                elif val == 9:
                    self.relative_base += op1

                self.index += 1

            elif val in [5, 6, 9]:
                op1 = self.read_val(mods, 0)

                if val == 5 and op1 or val == 6 and not op1:
                    self.index = self.read_val(mods, 1)
                else:
                    self.index += 2

            elif val == 99:
                break

            else:
                raise ValueError("RIP")

In [ ]:
seq = [109,424,203,1,21102,11,1,0,1106,0,282,21101,18,0,0,1106,0,259,2101,0,1,221,203,1,21102,31,1,0,1106,0,282,21102,1,38,0,1105,1,259,20102,1,23,2,22101,0,1,3,21101,0,1,1,21101,0,57,0,1106,0,303,1202,1,1,222,21001,221,0,3,20102,1,221,2,21102,259,1,1,21101,80,0,0,1105,1,225,21102,1,149,2,21101,0,91,0,1105,1,303,1202,1,1,223,21002,222,1,4,21102,259,1,3,21102,225,1,2,21102,225,1,1,21101,118,0,0,1105,1,225,20102,1,222,3,21101,0,127,2,21102,133,1,0,1105,1,303,21202,1,-1,1,22001,223,1,1,21102,1,148,0,1106,0,259,1201,1,0,223,21001,221,0,4,21002,222,1,3,21102,14,1,2,1001,132,-2,224,1002,224,2,224,1001,224,3,224,1002,132,-1,132,1,224,132,224,21001,224,1,1,21101,195,0,0,106,0,108,20207,1,223,2,20102,1,23,1,21101,0,-1,3,21102,214,1,0,1106,0,303,22101,1,1,1,204,1,99,0,0,0,0,109,5,1202,-4,1,249,22102,1,-3,1,21201,-2,0,2,21201,-1,0,3,21102,1,250,0,1105,1,225,22102,1,1,-4,109,-5,2106,0,0,109,3,22107,0,-2,-1,21202,-1,2,-1,21201,-1,-1,-1,22202,-1,-2,-2,109,-3,2105,1,0,109,3,21207,-2,0,-1,1206,-1,294,104,0,99,21202,-2,1,-2,109,-3,2106,0,0,109,5,22207,-3,-4,-1,1206,-1,346,22201,-4,-3,-4,21202,-3,-1,-1,22201,-4,-1,2,21202,2,-1,-1,22201,-4,-1,1,22101,0,-2,3,21101,343,0,0,1106,0,303,1106,0,415,22207,-2,-3,-1,1206,-1,387,22201,-3,-2,-3,21202,-2,-1,-1,22201,-3,-1,3,21202,3,-1,-1,22201,-3,-1,2,22101,0,-4,1,21102,1,384,0,1106,0,303,1105,1,415,21202,-4,-1,-4,22201,-4,-3,-4,22202,-3,-2,-2,22202,-2,-4,-4,22202,-3,-2,-3,21202,-4,-1,-2,22201,-3,-2,1,22102,1,1,-4,109,-5,2106,0,0]
intcode = IntCode(seq)

In [ ]:
import numpy as np
np.set_printoptions(linewidth =np.inf,threshold=np.inf)

dist = 50

s = 0

rep = np.empty((dist, dist), dtype=str)

for i in range(dist):
    for j in range(dist):
        intcode.input_val.extend([i, j])        
        output = list(intcode.run())[0]
        rep[i, j] = '#' if output else '.'
        s += output
        intcode.reset()

print(s)

In [ ]:

from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
rep

In [ ]:
def check_point(x, y):
    intcode.input_val.extend([x, y])
    inside = list(intcode.run())[0] == 1
    intcode.reset()
    return inside

In [ ]:
check_point(8, 9) # first point

In [ ]:

size = 100
x = 8
y = 9
while not check_point(x + size - 1, y - size + 1):
    y+=1
    while not check_point(x, y):
        x+=1
    
print(x*10000 + y-size+1)
    
        